In [ ]:
# this file includes loading file and creating dateset(shuffle, ) for iterator
# and this file is kind of like tensorflow.contrib.data.Dataset module
# and the txt_file should include the image path and image label in each line
import tensorflow as tf
import numpy as np
from tensorflow.contrib,.data import dataset
from tensorflow.python.framework import dtypes
from tensorflow.python.framework.ops import convert_to_tensor

ImageMean = tf.constant([123.68, 116.779, 103.939], dtype = tf.float32)

class ImageDataGenerator(object):
    def __init__(self, txt_file, batch_size, num_classes, 
                 shuffle = True, buffer_size = 1000):
    """
    Args:
        txt_file: file path for imagedata
         
    """  
    # file path and class number 
    self.txt_file = txt_file
    self.num_classes = num_classes
    # retrieve the iamegs and labels 
    self._read_txt_file()
    # get the number of samples
    self.num_samples = len(self.labels)
    
    # convert data to tensor and then use Dataset module
    self.img_paths = tf.convert_to_tensor(self.img_paths, dtype = tf.float32)
    self.labels = tf.convert_to_tensor(self.labels, dtype = tf.float32)
    # dataset is converted to (img_paths, img_labels)
    data = Dataset.from_tensor_slices((self.img_paths, self.labels))
    # get img pixel and one_hot label
    data = data.map(self._get_image_label_tensor)
    
    # shuffle the dataset
    if shuffle:
        data = data.suffle(buffer_size = buffer_size).batch(batch_size)
    
    # create a new dataset with batch size
    self.data = data
    
    
    def _read_txt_file(self):
        self.img_path = []
        self.img_label = []
        with open(self.txt_file, 'r') as f:
            lines = f.readlines()
            for line in lines:
                items = line.split(' ')
                self.img_path.append(items[0])
                self.img_label.append(items[1])
    
    def _get_image_label_tensor(self, fliename, label):
        img_string = tf.read_file(filename)
        # decode_*** depends on the image type(gif, png, ...) chanenels depends on RGB or grescary
        # get the pixal of the image
        img_decoded = tf.image.decode_png(img_string, channels = 3)
        # the input of alexnet is 227x227
        img_resized = tf.image.resize_images(img_decoded, [227, 227])
        img_centered = tf.subtract(img_resized, ImageMean)
        # one_hot for label
        one_hot = tf.one_hot(label, self.num_classes)
        
        # RGB->BGR the output is BGR with opencv
        # img = img_centered[:, :, ::-1]
        img = img_centered
        return img, one_hot
    